In [1]:
import cv2
import numpy as np
import time
import threading

from src.display import showarray
from src.feed import LiveFrameProcessor
from src.video import VideoHandler
from src.processing.analyzers import DiceAnalyzer

In [2]:
dice_mon = cv2.imread("data/images/rec_frame_c.jpg")

In [3]:
def get_dice_detector() -> cv2.SimpleBlobDetector:
    params = cv2.SimpleBlobDetector_Params()
    params.filterByArea = True
    params.minArea = 25
    params.maxArea = 60
    params.filterByCircularity = True
    params.minCircularity = 0.4
    params.filterByColor = True
    params.filterByConvexity = True
    params.minConvexity = 0.8
    params.filterByInertia = True
    params.minInertiaRatio = 0.7
    # params.minThreshold = 10
    # params.maxThreshold = 200
    # params.minInertiaRatio = 0.7
    return cv2.SimpleBlobDetector_create(params)

dice_detector = get_dice_detector()

In [4]:
def dilated_black_color_image(image: np.ndarray, threshold: int = 40) -> np.ndarray:
    gs_image = np.array(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
    gs_image[gs_image > threshold] = 255
    gs_image[gs_image <= threshold] = 0
    return gs_image

def detect_dice_dots(image: np.ndarray) -> list[cv2.KeyPoint]:
    return dice_detector.detect(dilated_black_color_image(image))

In [5]:
analyzer = DiceAnalyzer(dice_detector, dilated_black_color_image, threaded=True)
processor = LiveFrameProcessor("feed", 400, 800)
with VideoHandler("data/recordings/Easy_3.mp4") as vh:
    vh.go_through_video(processor, [analyzer])